In [2]:
import io
import os
import re
import requests
import pandas as pd

In [9]:
currency = "JPY"
base_url_cash = "https://rate.bot.com.tw/xrt/flcsv/0/{}/{}/1?lang=en-US"
os.makedirs("eur_data", exist_ok=True)

cash_list = []
spot_list = []

# 下載區間設定
start_year, start_month = 2024, 1
end_year, end_month = 2025, 8

for year in range(start_year, end_year+1):
    for month in range(1, 13):
        if (year > end_year) or (year == end_year and month > end_month):
            break
        yyyymm = f"{year}-{month:02d}"
        fn_prefix = f"{year}{month:02d}"

        # --- 下載 Cash (已含 Spot) ---
        url_cash = base_url_cash.format(yyyymm,currency)
        r = requests.get(url_cash)
        if r.ok and "Data Date" in r.text:
            content = r.text.replace('\ufeff', '').replace('ï»¿', '')
            # 讀入 dataframe
            data = pd.read_csv(io.StringIO(content)).astype(str)
            data = data[data.columns[:-1]]
            date = pd.to_datetime(data["Data Date"]) + pd.Timedelta(hours=12)
            # --- Cash 匯率 ---
            cash = data.loc[:,["Data Date","Cash","Cash.1"]]
            cash.columns = ["Date","Buying","Selling"]
            cash["Date"] = date
            cash_list.append(cash)
            # --- Spot 匯率 ---
            spot = data.loc[:,["Data Date","Spot","Spot.1"]]
            spot.columns = ["Date","Buying","Selling"]
            spot["Date"] = date
            spot_list.append(spot)

# 合併所有月份資料、排序
if cash_list:
    all_cash = pd.concat(cash_list, ignore_index=True)
    all_cash = all_cash.sort_values("Date").reset_index(drop=True)
    all_cash.to_csv(f"{currency}_Cash_Historical.csv",index=False)
else:
    all_cash = pd.DataFrame()

if spot_list:
    all_spot = pd.concat(spot_list, ignore_index=True)
    all_spot = all_spot.sort_values("Date").reset_index(drop=True)
    all_spot.to_csv(f"{currency}_Spot_Historical.csv",index=False)
else:
    all_spot = pd.DataFrame()


In [5]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fetch_twbank_exchange(target_currencies=['USD', 'JPY', 'EUR', 'CNY']):
    url = "https://rate.bot.com.tw/xrt/all/day"
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "html.parser")

    quoted = None
    info_p = soup.find("p", class_="text-info")
    if info_p and info_p.text:
        m = re.search(r'(\d{4}/\d{2}/\d{2} \d{2}:\d{2})', info_p.text)
        if m:
            quoted = m.group(1)
    if quoted is None:
        print("Warning： Cannot find Quoted Date. Replace with None")

    table = soup.find("table", class_="table")
    rows = table.find("tbody").find_all("tr")
    records = []

    for row in rows:
        cols = row.find_all("td")
        if not cols or len(cols) < 5:
            continue
        currency = cols[0].text.strip()
        for cur in target_currencies:
            if cur in currency:
                try:
                    record = {
                        "Date": pd.to_datetime(quoted),
                        "Currency": cur,
                        "Cash": float(cols[1].text.replace(',', '').strip()) if cols[1].text.strip() else None,
                        "Cash.1": float(cols[2].text.replace(',', '').strip()) if cols[2].text.strip() else None,
                        "Spot": float(cols[3].text.replace(',', '').strip()) if cols[3].text.strip() else None,
                        "Spot.1": float(cols[4].text.replace(',', '').strip()) if cols[4].text.strip() else None,
                    }
                    records.append(record)
                except Exception as e:
                    print(f"Parse error for {cur}: {e}")
                break

    df = pd.DataFrame(records, columns=["Date", "Currency", "Cash", "Cash.1", "Spot", "Spot.1"])
    return df.set_index("Currency")

if __name__ == "__main__":
    df = fetch_twbank_exchange()
    for t in ['Cash','Spot']:
        for currency in ["USD","JPY","EUR","CNY"]:
            path = f'{currency}_{t}_Historical.csv'
            df_hist = pd.read_csv(path)
            
            df_new = df.loc[[currency]][["Date", f"{t}", f"{t}.1"]].copy()
            df_new.columns = ["Date", "Buying", "Selling"]
            
            existing_dates = set(pd.to_datetime(df_hist["Date"]).dt.strftime('%Y-%m-%d %H:%M'))
            new_date_str = df_new["Date"].iloc[0].strftime('%Y-%m-%d %H:%M')
            if new_date_str not in existing_dates:
                df_all = pd.concat([df_hist, df_new], ignore_index=True)
                df_all["Date"] = pd.to_datetime(df_all["Date"])
                df_all = df_all.sort_values("Date").reset_index(drop=True)
                df_all.to_csv(path, index=False)
                print(f"New data：{currency}_{t} {new_date_str}")
            else:
                print(f"{currency}_{t}: {new_date_str} Existing")


USD_Cash: 2025-08-04 16:02 Existing
New data：JPY_Cash 2025-08-04 16:02
EUR_Cash: 2025-08-04 16:02 Existing
CNY_Cash: 2025-08-04 16:02 Existing
USD_Spot: 2025-08-04 16:02 Existing
New data：JPY_Spot 2025-08-04 16:02
EUR_Spot: 2025-08-04 16:02 Existing
CNY_Spot: 2025-08-04 16:02 Existing
